In [2]:
#pip install gradio

In [3]:
import gradio as gr

import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR

## Chargement des données

In [24]:
requests=pd.read_csv("/content/data/data.csv",sep=",")
#requests.index=requests['hotel_id']
#requests=requests.drop(['Unnamed: 0'], axis=1)
#requests=requests.drop(['Unnamed: 0.1'], axis=1)
#requests=requests.drop(['Unnamed: 0.1.1'], axis=1)
#requests=requests.drop(['Unnamed: 0.1.1.1'], axis=1)
#requests=requests.drop(['index'], axis=1)
#requests=requests.drop(['hotel_id'],axis=1)
#requests=requests.drop(['hotel_id.1'],axis=1)
requests.head()

,934,10934,70.0,317.0,317.0.1,Accar Hotels,Marcure,amsterdam,0,0.1,0.2,190,29,20,german,1,laulau
0,10935,10935,71.0,507.0,507.0,Boss Western,Boss Western,amsterdam,0,0,0,132,16,20,german,1,laulau
1,10936,10936,72.0,82.0,82.0,Independant,Independant,amsterdam,0,0,0,113,20,20,german,1,laulau
2,10937,10937,73.0,117.0,117.0,Yin Yang,8 Premium,amsterdam,1,0,0,78,12,20,german,1,laulau
3,10938,10938,74.0,557.0,557.0,Accar Hotels,Navatel,amsterdam,0,0,0,117,0,20,german,1,laulau
4,10939,10939,75.0,214.0,214.0,Chillton Worldwide,Quadrupletree,amsterdam,0,1,0,200,27,20,german,1,laulau


In [18]:
def good_format(requests):
  requests["pool"]=pd.Categorical(requests["pool"],ordered=False)
  requests["parking"]=pd.Categorical(requests["parking"],ordered=False)
  requests["children_policy"]=pd.Categorical(requests["children_policy"],ordered=False)
  requests["language"]=pd.Categorical(requests["language"],ordered=False)
  requests["mobile"]=pd.Categorical(requests["mobile"],ordered=False)
  requests["city"]=pd.Categorical(requests["city"],ordered=False)
  requests["group"]=pd.Categorical(requests["group"],ordered=False)
  requests["brand"]=pd.Categorical(requests["brand"],ordered=False)

  #on transforme les catégories de string en catégories de nombres (on pourra faire les correspondances plus tard)
  _, opt = np.unique(np.array(requests['group']), return_inverse=True)
  requests['group']=opt
  _, opt = np.unique(np.array(requests['brand']), return_inverse=True)
  requests['brand']=opt
  _, opt = np.unique(np.array(requests['city']), return_inverse=True)
  requests['city']=opt
  _, opt = np.unique(np.array(requests['language']), return_inverse=True)
  requests['language']=opt

  return requests

In [19]:
requests=good_format(requests)

TypeError: ignored

In [ ]:
requests.head()

,group,brand,city,parking,pool,children_policy,price,stock,date,language,mobile,avatar_name
hotel_id,,,,,,,,,,,,
0,4,4,0,1,1,0,366,14,10,0,0,laulau
1,0,14,0,1,0,0,359,10,10,0,0,laulau
2,5,1,0,1,1,0,193,10,10,0,0,laulau
3,1,2,0,0,0,0,205,4,10,0,0,laulau
4,0,6,0,1,0,0,86,0,10,0,0,laulau


## Création de l'échantillon d'apprentissage et de l'échantillon test

In [25]:
#j'ai utilisé un jeu de données plus petit pour pouvoir entrainer mon modèle plus vite
requests=pd.read_csv("/content/data/laulau_data.csv",sep=",",header=0)
requests.index=requests['hotel_id']
requests=requests.drop(['Unnamed: 0'], axis=1)
requests=requests.drop(['index'], axis=1)
requests=requests.drop(['hotel_id'],axis=1)
# je drop l'avatar pour le test car ici on en a qu'un 
requests=requests.drop(['avatar_name'], axis=1)
requests.head()

,group,brand,city,parking,pool,children_policy,price,stock,date,language,mobile
hotel_id,,,,,,,,,,,
519,Yin Yang,Royal Lotus,copenhagen,0,0,0,208,68,30,french,0
523,Independant,Independant,copenhagen,0,0,0,85,41,30,french,0
898,Yin Yang,8 Premium,copenhagen,1,0,0,86,24,30,french,0
770,Morriott International,Morriot,copenhagen,1,1,0,210,73,30,french,0
562,Chillton Worldwide,Quadrupletree,copenhagen,0,0,0,215,74,30,french,0


In [26]:
requests=good_format(requests)
requests.head()

,group,brand,city,parking,pool,children_policy,price,stock,date,language,mobile
hotel_id,,,,,,,,,,,
519,5,13,0,0,0,0,208,68,30,4,0
523,3,7,0,0,0,0,85,41,30,4,0
898,5,0,0,1,0,0,86,24,30,4,0
770,4,10,0,1,1,0,210,73,30,4,0
562,2,12,0,0,0,0,215,74,30,4,0


In [27]:
price=requests["price"]
X_train,X_test,price_train,price_test=train_test_split(requests,price,test_size=0.25,random_state=11)
X_train=X_train.drop(['price'], axis=1)
X_test=X_test.drop(['price'], axis=1)

In [28]:
X_train.head()

,group,brand,city,parking,pool,children_policy,stock,date,language,mobile
hotel_id,,,,,,,,,,
237,4,5,2,1,0,0,56,30,2,0
476,3,7,2,1,0,0,33,30,2,0
221,3,7,3,1,0,0,28,30,1,0
618,0,11,1,1,0,0,39,30,0,0
600,3,7,1,1,0,0,27,30,3,0


# Prédiction du prix des hôtels avec plusieurs méthodes

In [29]:
# On définit la fonction MAPE : Mean Absolute Percentage Error
def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

In [ ]:
#on cherche les meilleurs paramètres puis on entraine le modèle avec ces derniers
rate = list(1e-1*np.array(list(range(1,20,3))))
acti = ['relu','tanh','logistic','identity']
#hidden = list(range(10,150,10))
hidden = list(range(10,60,10))
param_grid=[{"hidden_layer_sizes":hidden,"activation":acti,"learning_rate_init":rate}]
nnet= GridSearchCV(MLPRegressor(max_iter=500),param_grid,cv=10,n_jobs=-1)
nnetOpt=nnet.fit(X_train, price_train)
# paramètre optimal
print("Meilleur score = %f, Meilleur paramètre = %s" % (1. - nnetOpt.best_score_,nnetOpt.best_params_))

## Calcul de l'erreur de prédiction

In [34]:
estimated_price_nnet=nnetOpt.predict(X_test)

### MSE

In [35]:
mean_squared_error(price_test, estimated_price_nnet)

848.6890242638823

### MAPE

In [ ]:
MAPE(price_test,estimated_price_nnet)

17.157881313487046

### Utilisation de Gradio

il faut trouver un moyen pour que le nom de la ville/brand/group/language rentré par l'utilisateur dans gradio soit convertie en un chiffre qui est identique à celui qui est utilisé dans les prédictions dans l'étape précédente.

In [ ]:
def predict_price(group,brand,city,language,date,parking,pool,mobile, children_policy,stock):
  hotel_features = pd.DataFrame(columns=X_test.columns)
  if parking: 
    park=1
  else:
    park=0

  if pool:
    pool=1
  else:
    pool=0

  if children_policy=="no_policy":
    children_policy=0
  elif children_policy=="no_less_12":
    children_policy=1
  else:
    children_policy=2

  print(children_policy)
  
  if mobile:
    mobile=1
  else:
    mobile=0
  
  hotel_features=hotel_features.append({'group' : group, 
                'brand' : brand, 
                'city' : city, 
                "parking" : park,
               "pool" : pool,
                "children_policy" : children_policy,
                "stock" : stock,
                "date" : date,
                "language" : language,
                "mobile" : mobile
                }
                , ignore_index=True)
  
  hotel_features=good_format(hotel_features)

  estimated_price_nnet=nnetOpt.predict(hotel_features)
  print(estimated_price_nnet)
  #return hotel_features
  return int(estimated_price_nnet),hotel_features

In [ ]:
predict_price("madrid","laurie","toulouse","fjfjf",12,1,1,1,"coucou",12)

2
[89.44175282]


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:566: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  X = check_array(X, **check_params)


(89,
    group  brand  city parking pool children_policy stock date  language mobile
 0      0      0     0       1    1               2    12   12         0      1)

In [ ]:
date = gr.inputs.Slider(minimum=1, maximum=44, step=1)
children_policy = gr.inputs.Radio(choices=["no_policy", "no_less_12", "no_children"])

demo = gr.Interface(
    fn=predict_price,
    inputs=["text", "text","text","text",date,"checkbox","checkbox","checkbox",children_policy,"number"],
    outputs=["number","text"],
)
demo.launch()

/usr/local/lib/python3.7/dist-packages/gradio/inputs.py:89: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  "Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components",
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/inputs.py:183: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  "Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components",


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2280e6376ce24407.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7f18936a6a90>,
 'http://127.0.0.1:7868/',
 'https://2280e6376ce24407.gradio.app')